In [67]:
from neo4j import GraphDatabase, basic_auth
import nxneo4j as nx #using nxneo4j
from py2neo import Graph, Node 
import pandas

In [68]:
cGraph = Graph("neo4j+s://101fd6b7.databases.neo4j.io", auth=('neo4j', "gB9F-fD2doYqInIcXR3DJZwnvvDWm-ZpgvOJ3BGCl54"))

In [87]:
recipeList = pandas.read_csv('../CSV/recipeList.csv')
listOfRecipes = pandas.read_csv('../CSV/recipeLinks.csv')

In [88]:
uniqueIng = []

for index, row in recipeList.iterrows():
    temp = [row[1].replace("_", " "), row[2]]
    if temp not in uniqueIng:
        uniqueIng.append(temp)
        

In [89]:
uniqueRecipe = []
uniqueRecipeAndLink = []

for index, row in listOfRecipes.iterrows():
    temp = [row[1], row[2]]
    if row[1] not in uniqueRecipe:
        uniqueRecipe.append(row[1])
        uniqueRecipeAndLink.append(temp)

In [112]:
for currIng, ingType in uniqueIng:
    currIng = currIng.replace("'", "-")
    addNode = "CREATE (n:Ingredient {name : '" + currIng + "', type: '" + str(ingType) + "'})"
        
    cGraph.query(addNode)

In [113]:
for recipe in uniqueRecipeAndLink:
    addNode = "CREATE (n:Recipe {name : '" + recipe[0].replace("'", "-") + "', link : '" + recipe[1] + "'})"
    cGraph.query(addNode)

In [106]:
ingPairWeights = pandas.read_csv('../CSV/ingPairWeights.csv')

In [107]:
counter = 0
tempArray = []
splitIngPairWeights = []
for index, row in ingPairWeights.iterrows():
    if counter == 3000:
        splitIngPairWeights.append(tempArray)
        tempArray = []
        counter = 0

    tempArray.append(row)
    counter += 1

splitIngPairWeights.append(tempArray)

In [144]:
counter = 0
c = 12
for i in range(0, len(splitIngPairWeights[c])):
    if splitIngPairWeights[c][i][2] != 0 or splitIngPairWeights[c][i][3] != 0:
        counter += 1

counter

2112

In [145]:
iterCounter = 12 # change to the next index before you run (last run was done on 12) (up to 12)

for counter in range(0, len(splitIngPairWeights[iterCounter])): 
    splitIngPairWeights[iterCounter][counter][0] = splitIngPairWeights[iterCounter][counter][0].replace("'", "-").replace("_", " ")
    splitIngPairWeights[iterCounter][counter][1] = splitIngPairWeights[iterCounter][counter][1].replace("'", "-").replace("_", " ")

    if splitIngPairWeights[iterCounter][counter][2] != 0 or splitIngPairWeights[iterCounter][counter][3] != 0:
        addEdge = "MATCH (a:Ingredient), (b:Ingredient) WHERE a.name = '" + splitIngPairWeights[iterCounter][counter][0] + "' AND b.name = '" + splitIngPairWeights[iterCounter][counter][1] + "' CREATE (a)-[r:IngPair {comRecWeight:" + str(splitIngPairWeights[iterCounter][counter][2]) + ", comCompWeight:" + str(splitIngPairWeights[iterCounter][counter][3]) + "}]->(b) RETURN type(r)"
        cGraph.query(addEdge)

In [115]:
recipeIngredients = []

for counter in range(0, len(recipeList)):
    prepIngredient = recipeList['Ingredient'][counter].replace("'", "-").replace("_", " ")
    prepRecipe = recipeList['Recipe'][counter].replace("'", "-").replace("_", " ")
    entry = [prepRecipe, prepIngredient]
    if entry not in recipeIngredients:
        recipeIngredients.append(entry)

print(len(recipeIngredients))

1636


In [116]:
result = []
tempResult = []
for entry in recipeIngredients:
    addEdge = "MATCH (a:Recipe), (b:Ingredient) WHERE a.name = '" + entry[0] + "' AND b.name = '" + entry[1] + "' CREATE (a)-[r:RecIng]->(b) RETURN type(r)"
    temp = cGraph.query(addEdge)
    tempResult = [entry, temp.data()]
    result.append(tempResult)